In [9]:
import pandas as pd
import importlib
import YandexGPT_API
importlib.reload(YandexGPT_API)
import chromadb
from chromadb.config import Settings
import json

In [10]:
df_news = pd.read_csv('news_cut.csv').head(50)
df_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   url     50 non-null     object
 1   title   50 non-null     object
 2   text    50 non-null     object
 3   topic   47 non-null     object
 4   tags    49 non-null     object
 5   date    50 non-null     object
dtypes: object(6)
memory usage: 2.5+ KB


In [11]:
df_news.drop(['title', 'topic', 'tags', 'date'], axis=1, inplace=True)

In [12]:
df_news.head(5)

,url,text
0,https://lenta.ru/news/2006/06/25/marketreview/,"Индекс РТС на торгах 23 июня опустился на 0,68..."
1,https://lenta.ru/news/2001/09/04/ntv/,"Бизнесмен и солист группы ""Белый орел"" Владими..."
2,https://lenta.ru/news/2009/03/30/monsters/,"Анимационный фильм ""Монстры против пришельцев""..."
3,https://lenta.ru/news/2018/01/04/valdes/,Чемпион мира и Европы в составе сборной Испани...
4,https://lenta.ru/news/2016/08/25/ninaeremina/,Чемпионка мира по баскетболу в составе сборной...


In [14]:
gpt = YandexGPT_API.YandexGPTEmbeddings()

In [15]:
message = 'Кто проживает на дне океана'
answer = gpt.embed_document(message)

06-06-2024 15:10:32 - urllib3.connectionpool - DEBUG @ Starting new HTTPS connection (1): llm.api.cloud.yandex.net:443
06-06-2024 15:10:32 - urllib3.connectionpool - DEBUG @ https://llm.api.cloud.yandex.net:443 "POST /foundationModels/v1/textEmbedding HTTP/1.1" 200 None


<Response [200]>


In [ ]:
answers = gpt.embed_documents(df_news['text'])

In [ ]:
answers

In [ ]:
df_news['embeds'] = answers
df_news.to_csv('news_embed.csv', index=False)

In [ ]:
chroma_client = chromadb.HttpClient(host='localhost', 
                                    port="8000", 
                                    settings=Settings(anonymized_telemetry=False))

collection = chroma_client.get_or_create_collection("news")

In [ ]:
df = pd.read_csv('news_embed.csv')
texts = df["text"].tolist()

In [ ]:
text_embeddings =  list(map(json.loads, df["embeds"].tolist()))
ids = df_news.index.astype(str).tolist()

In [ ]:
collection.upsert(
    ids=ids,
    embeddings=text_embeddings,
    metadatas=[{"source": "news", "text": txt} for txt in texts],
    documents=texts
)